# Table of Contents
- [Optimizing Model Parameter](#scrollTo=Ep7xLMticUYB&line=6&uniqifier=1)
    - [Hyperparameters](#scrollTo=Yv2bhDbMdkbQ&line=8&uniqifier=1)
    - [Optimization Loop](#scrollTo=ThDjKHpceEOq&line=6&uniqifier=1)
    - [Loss Function](#scrollTo=NJmzKDateY7x&line=6&uniqifier=1)
    - [Optimizer](#scrollTo=ILpsWmOOfFRQ&line=4&uniqifier=1)
    - [Full Implementation](#scrollTo=oCuLF3O1gLlQ&line=2&uniqifier=1)

# Optimizing Model Parameter
Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data. 

**Training a model is an iterative process**; in each iteration (called an **epoch**) the model makes a guess about the output, calculates the error in its guess (**loss**), collects the derivatives of the error with respect to its parameters, and optimizes these parameters using gradient descent. 

For a more detailed walkthrough of this process, check out this video on [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

In [1]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root='./data',
    train=True,
    transform=ToTensor(),
    download=True,
)

test_data = datasets.FashionMNIST(
    root='./data',
    train=False,
    transform=ToTensor(),
    download=True,
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters
Hyperparameters
Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates.

We define the following hyperparameters for training:
1. **Number of Epochs** - the number times to iterate over the dataset
2. **Batch Size** - the number of data samples propagated through the network before the parameters are updated
3. **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [5]:
learning_rate = 1e-3
batch_size = 64
epoch = 5

## Optimization Loop
Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.

Each epoch consists of two main parts:
1. **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.
2. **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

## Loss Function
When presented with some training data, our untrained network is likely not to give the correct answer. **Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training**. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include `nn.MSELoss` (Mean Square Error) for regression tasks, and `nn.NLLLoss` (Negative Log Likelihood) for classification. `nn.CrossEntropyLoss` **combines** `nn.LogSoftmax` and `nn.NLLLoss.`

We pass our model’s output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error.

In [7]:
# initialize loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizier
Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the `optimizer` object. Here, we use the **SGD optimizer**; additionally, there are [many different](https://pytorch.org/docs/stable/optim.html) optimizers available in PyTorch such as **ADAM** and **RMSProp**, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [9]:
# initialize the optimizer
# lr stands for learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
1. Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
2. Backpropagate the prediction loss with a call to `loss.backwards()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
3. Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation
We define `train_loop` that loops over our optimization code, and `test_loop` that evaluates the model’s performance against our test data.

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to `train_loop` and `test_loop`. Feel free to increase the number of epochs to track the model’s improving performance.

In [11]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305075  [    0/60000]
loss: 2.290163  [ 6400/60000]
loss: 2.267820  [12800/60000]
loss: 2.263571  [19200/60000]
loss: 2.263963  [25600/60000]
loss: 2.227719  [32000/60000]
loss: 2.236290  [38400/60000]
loss: 2.200539  [44800/60000]
loss: 2.206075  [51200/60000]
loss: 2.167361  [57600/60000]
Test Error: 
 Accuracy: 38.8%, Avg loss: 2.163508 

Epoch 2
-------------------------------
loss: 2.177363  [    0/60000]
loss: 2.163083  [ 6400/60000]
loss: 2.101506  [12800/60000]
loss: 2.121811  [19200/60000]
loss: 2.081985  [25600/60000]
loss: 2.015881  [32000/60000]
loss: 2.053370  [38400/60000]
loss: 1.970169  [44800/60000]
loss: 1.982386  [51200/60000]
loss: 1.897741  [57600/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.901539 

Epoch 3
-------------------------------
loss: 1.939704  [    0/60000]
loss: 1.903115  [ 6400/60000]
loss: 1.785831  [12800/60000]
loss: 1.830496  [19200/60000]
loss: 1.723891  [25600/60000]
loss: 1.672582  [32000/600